In [2]:
import pandas as pd
import numpy as np

Dataframe yang dibuat:

DataFrame products: Berisi data produk dengan 100 entri unik. Setiap produk memiliki Product_ID, harga (Price) yang dihasilkan secara acak antara 10 hingga 500, biaya (Cost) yang dihasilkan secara acak antara 5 hingga 300, dan Supplier_ID yang dipilih secara acak dari 1 hingga 10.

DataFrame sales: Menyimpan data penjualan dengan 1000 transaksi. Setiap transaksi mencatat Product_ID yang diambil secara acak dari DataFrame products, jumlah unit terjual (Units_Sold) yang dihasilkan secara acak antara 1 hingga 10, dan tanggal penjualan (Date_Sold) yang mencakup rentang 1000 hari mulai dari 1 Januari 2024.

DataFrame suppliers: Memuat informasi tentang pemasok dengan 10 entri unik. Setiap pemasok memiliki Supplier_ID, nama (Supplier_Name) yang diformat sebagai “Supplier_i” di mana i adalah nomor urut pemasok, dan skor keandalan (Reliability_Score) yang dihasilkan secara acak antara 0 hingga 1.

DataFrame merged_data: Hasil penggabungan DataFrame sales, products, dan suppliers berdasarkan Product_ID dan Supplier_ID. DataFrame ini memungkinkan analisis terpadu yang mencakup semua aspek dari produk, penjualan, dan informasi pemasok.

In [5]:
# Misalkan kita memiliki data produk, penjualan, dan pemasok
products = pd.DataFrame({
    'Product_ID': np.arange(1, 101),
    'Price': np.random.uniform(10, 500, 100).round(2),
    'Cost': np.random.uniform(5, 300, 100).round(2),
    'Supplier_ID': np.random.choice(np.arange(1, 11), 100)
})

sales = pd.DataFrame({
    'Product_ID': np.random.choice(products['Product_ID'], 1000),
    'Units_Sold': np.random.randint(1, 10, 1000),
    'Date_Sold': pd.date_range(start='2024-01-01', periods=1000, freq='D')
})

suppliers = pd.DataFrame({
    'Supplier_ID': np.arange(1, 11),
    'Supplier_Name': [f'Supplier_{i}' for i in range(1, 11)],
    'Reliability_Score': np.random.uniform(0, 1, 10).round(2)
})

# Menggabungkan data untuk analisis
merged_data = pd.merge(pd.merge(sales, products, on='Product_ID'), suppliers, on='Supplier_ID')
merged_data

,Product_ID,Units_Sold,Date_Sold,Price,Cost,Supplier_ID,Supplier_Name,Reliability_Score
0,53,1,2024-01-01,270.89,226.36,7,Supplier_7,0.72
1,53,8,2024-01-08,270.89,226.36,7,Supplier_7,0.72
2,53,4,2024-01-19,270.89,226.36,7,Supplier_7,0.72
3,53,3,2024-04-04,270.89,226.36,7,Supplier_7,0.72
4,53,1,2024-06-22,270.89,226.36,7,Supplier_7,0.72
...,...,...,...,...,...,...,...,...
995,13,8,2025-10-18,89.48,11.74,3,Supplier_3,0.53
996,13,1,2025-11-21,89.48,11.74,3,Supplier_3,0.53
997,13,5,2026-03-27,89.48,11.74,3,Supplier_3,0.53
998,13,1,2026-06-07,89.48,11.74,3,Supplier_3,0.53


In [7]:
# Analisis Optimasi Inventori
# Menghitung rata-rata penjualan harian per produk
daily_sales_avg = merged_data.groupby('Product_ID')['Units_Sold'].mean()
daily_sales_avg

Product_ID
1      3.800000
2      3.833333
3      4.250000
4      4.526316
5      3.857143
         ...   
96     4.266667
97     5.000000
98     4.818182
99     5.166667
100    4.000000
Name: Units_Sold, Length: 100, dtype: float64

In [8]:
# Menghitung safety stock dengan asumsi lead time dan service level tertentu
lead_time = 7  # dalam hari
service_level = 0.95  # 95% service level
z_score = 1.645  # z-score untuk 95% service level
safety_stock = {}
for product_id, avg in daily_sales_avg.items():
    std_dev = merged_data[merged_data['Product_ID'] == product_id]['Units_Sold'].std()
    safety_stock[product_id] = round(z_score * std_dev * np.sqrt(lead_time))

In [12]:
# Analisis Biaya
# Menghitung total biaya dan keuntungan
merged_data['Total_Cost'] = merged_data['Units_Sold'] * merged_data['Cost']
merged_data['Total_Revenue'] = merged_data['Units_Sold'] * merged_data['Price']
merged_data['Profit'] = merged_data['Total_Revenue'] - merged_data['Total_Cost']
total_profit = merged_data['Profit'].sum()

print('Total Biaya')
print(merged_data['Total_Cost'])
print('')
print('Total Pendapatan')
print(merged_data['Total_Revenue'])
print('')
print('Total Keuntungan')
print(merged_data['Profit'])
print('')
print('Total keuntungannya adalah:')
print(total_profit)

Total Biaya
0       226.36
1      1810.88
2       905.44
3       679.08
4       226.36
        ...   
995      93.92
996      11.74
997      58.70
998      11.74
999      11.74
Name: Total_Cost, Length: 1000, dtype: float64

Total Pendapatan
0       270.89
1      2167.12
2      1083.56
3       812.67
4       270.89
        ...   
995     715.84
996      89.48
997     447.40
998      89.48
999      89.48
Name: Total_Revenue, Length: 1000, dtype: float64

Total Keuntungan
0       44.53
1      356.24
2      178.12
3      133.59
4       44.53
        ...  
995    621.92
996     77.74
997    388.70
998     77.74
999     77.74
Name: Profit, Length: 1000, dtype: float64

Total keuntungannya adalah:
410301.95999999996


In [13]:
# Forecasting Permintaan
# Misal kita menggunakan model sederhana seperti moving average
merged_data['Date_Sold'] = pd.to_datetime(merged_data['Date_Sold'])
merged_data.set_index('Date_Sold', inplace=True)
forecast = merged_data['Units_Sold'].rolling(window=30).mean().iloc[-1]

In [14]:
# Evaluasi Kinerja Pemasok
# Menggunakan reliability score
supplier_performance = suppliers.set_index('Supplier_ID')['Reliability_Score']

In [15]:
# Interpretasi hasil
print(f"Safety stock per produk: {safety_stock}")
print(f"Total keuntungan: {total_profit}")
print(f"Forecast permintaan terakhir: {forecast}")
print(f"Evaluasi kinerja pemasok: {supplier_performance}")

Safety stock per produk: {1: 9, 2: 11, 3: 10, 4: 11, 5: 11, 6: 11, 7: 11, 8: 14, 9: 10, 10: 8, 11: 13, 12: 14, 13: 13, 14: 12, 15: 14, 16: 3, 17: 10, 18: 8, 19: 11, 20: 12, 21: 8, 22: 15, 23: 14, 24: 13, 25: 11, 26: 11, 27: 15, 28: 11, 29: 11, 30: 11, 31: 8, 32: 10, 33: 13, 34: 9, 35: 11, 36: 12, 37: 12, 38: 12, 39: 14, 40: 9, 41: 12, 42: 10, 43: 10, 44: 14, 45: 10, 46: 13, 47: 15, 48: 8, 49: 11, 50: 11, 51: 11, 52: 13, 53: 12, 54: 9, 55: 8, 56: 10, 57: 11, 58: 14, 59: 9, 60: 14, 61: 7, 62: 11, 63: 12, 64: 9, 65: 11, 66: 15, 67: 10, 68: 14, 69: 8, 70: 11, 71: 9, 72: 13, 73: 12, 74: 7, 75: 13, 76: 10, 77: 10, 78: 11, 79: 13, 80: 12, 81: 10, 82: 11, 83: 13, 84: 11, 85: 13, 86: 11, 87: 11, 88: 12, 89: 14, 90: 12, 91: 12, 92: 11, 93: 8, 94: 6, 95: 12, 96: 14, 97: 9, 98: 11, 99: 15, 100: 12}
Total keuntungan: 410301.95999999996
Forecast permintaan terakhir: 4.466666666666667
Evaluasi kinerja pemasok: Supplier_ID
1     0.06
2     0.78
3     0.53
4     0.05
5     0.51
6     0.11
7     0.72
8 